# Benchmarking

# Elo benchmarking

In [1]:
import helpers
import pandas as pd
import os
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score,  balanced_accuracy_score
import json

TEST_SIZE_PERCENT = 0.2
PATH_ELO_CLUBS = './data/elo/elo-clubs/GER/'
HOME_TEAM_WIN = 2
AWAY_TEAM_WIN = 0
DRAW = 1
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [2]:
df1 = pd.read_pickle("./df_merge_club_transfer_data_prep1.pkl")
df2 = df1.copy()

In [3]:
# Kombinieren die Werte beider Spalten in eine Serie
combined_teams = pd.concat([df2['HOME_TEAM'], df2['AWAY_TEAM']])

# Initialisieren und anpassen Sie den LabelEncoder
le_teams = LabelEncoder()
le_teams.fit(combined_teams)

# Codieren Sie die 'HOME_TEAM' und 'AWAY_TEAM' Spalten
df2['HOME_TEAM'] = le_teams.transform(df2['HOME_TEAM'])
df2['AWAY_TEAM'] = le_teams.transform(df2['AWAY_TEAM'])

# Erstellen ein Mapping-Dictionary für die Decodierung
label_mapping_TEAM = {idx: label for idx, label in enumerate(le_teams.classes_)}

print("Label Mapping:", label_mapping_TEAM)

Label Mapping: {0: '1.FC Kaiserslautern', 1: '1.FC Köln', 2: '1.FC Nürnberg', 3: '1.FC Union Berlin', 4: '1.FSV Mainz 05', 5: 'Alemannia Aachen', 6: 'Arminia Bielefeld', 7: 'Bayer 04 Leverkusen', 8: 'Borussia Dortmund', 9: 'Borussia Mönchengladbach', 10: 'Eintracht Braunschweig', 11: 'Eintracht Frankfurt', 12: 'FC Augsburg', 13: 'FC Bayern München', 14: 'FC Energie Cottbus', 15: 'FC Hansa Rostock', 16: 'FC Ingolstadt 04', 17: 'FC Schalke 04', 18: 'FC St. Pauli', 19: 'Fortuna Düsseldorf', 20: 'Hamburger SV', 21: 'Hannover 96', 22: 'Hertha BSC', 23: 'Karlsruher SC', 24: 'MSV Duisburg', 25: 'RasenBallsport Leipzig', 26: 'SC Freiburg', 27: 'SC Paderborn 07', 28: 'SV Darmstadt 98', 29: 'SV Werder Bremen', 30: 'SpVgg Greuther Fürth', 31: 'TSG 1899 Hoffenheim', 32: 'VfB Stuttgart', 33: 'VfL Bochum', 34: 'VfL Wolfsburg'}


In [4]:
le_RESULT = LabelEncoder()
encoded_RESULT = le_RESULT.fit_transform(df2['RESULT'])
df2['RESULT'] = encoded_RESULT

label_mapping_RESULT = {idx: label for idx, label in enumerate(le_RESULT.classes_)}
print("Label Mapping:", label_mapping_RESULT)

Label Mapping: {0: 'AWAY_WIN', 1: 'DRAW', 2: 'HOME_WIN'}


In [5]:
selected_features = [
"DATE", "HOME_TEAM", "PLACE_HOME_TEAM", "HOME_PLAYERS_COUNT", "HOME_PLAYERS_AVG_AGE", "HOME_LEGIONARIES_COUNT",
"HOME_AVG_MARKET_VALUE", "HOME_TOTAL_MARKET_VALUE", "HOME_AVG_AGE_JOINING", "HOME_AVG_AGE_LEAVING", 
"AWAY_TEAM", "PLACE_AWAY_TEAM", "AWAY_PLAYERS_COUNT", "AWAY_PLAYERS_AVG_AGE", "AWAY_LEGIONARIES_COUNT",
"AWAY_AVG_MARKET_VALUE", "AWAY_TOTAL_MARKET_VALUE", "AWAY_AVG_AGE_JOINING", "AWAY_AVG_AGE_LEAVING", 
"MATCHDAY"
]

In [6]:
X = df2[selected_features]
y = df2['RESULT']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TEST_SIZE_PERCENT, shuffle=False)

X_test = X_test.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)

In [7]:
print(type(y_test))
print(len(y_test))
print(y_test.iloc[0])

<class 'pandas.core.series.Series'>
1163
2


In [8]:
print(len(X_test))
print(X_test['DATE'].iloc[0])

1163
2019-10-06 00:00:00


In [9]:
warnings.filterwarnings("ignore", category=UserWarning, message=".*Precision and F-score are ill-defined.*") 
results = helpers.get_elo_forecast_results(X_test, y_test, label_mapping_TEAM, PATH_ELO_CLUBS, HOME_TEAM_WIN, AWAY_TEAM_WIN, DRAW, le_teams)
with open('./results/elo_results.json', 'w') as f:
    json.dump(results, f)

Total Accuracy: 0.4961580680570801
